In [1]:
#To allocate memorey for gpu(here we are using 50%)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [4]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential

In [5]:
import numpy as np
from glob import glob

In [6]:
#resize all images
IMAGE_SIZE = [224,224]
train_path = 'train'
test_path = 'test'

In [7]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)

In [8]:
#dont train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [9]:
train_folders = glob('train/*')
train_folders

['train\\covid', 'train\\normal']

In [10]:
test_folders = glob('test/*')
test_folders

['test\\covid', 'test\\normal']

In [11]:
#flattern layer
x = Flatten()(vgg16.output)

In [12]:
prediction = Dense(2,activation='softmax')(x)
#create model
model = Model(inputs=vgg16.input,outputs=prediction)

In [13]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True)

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
training_set = train_datagen.flow_from_directory('train',
                                                 target_size=(224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 11043 images belonging to 2 classes.


In [18]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2765 images belonging to 2 classes.


In [19]:
#fit the model
r = model.fit_generator(training_set,
                        validation_data = test_set,
                        epochs=20,
                        steps_per_epoch = len(training_set),
                        validation_steps = len(test_set))

Epoch 1/20
346/346 [==============================] - 720s 2s/step - loss: 0.2855 - accuracy: 0.8812 - val_loss: 0.0601 - val_accuracy: 0.9414
Epoch 2/20
346/346 [==============================] - 140s 405ms/step - loss: 0.1858 - accuracy: 0.9256 - val_loss: 0.0178 - val_accuracy: 0.9316
Epoch 3/20
346/346 [==============================] - 177s 513ms/step - loss: 0.1582 - accuracy: 0.9373 - val_loss: 0.0948 - val_accuracy: 0.9421
Epoch 4/20
346/346 [==============================] - 118s 341ms/step - loss: 0.1430 - accuracy: 0.9435 - val_loss: 0.1408 - val_accuracy: 0.9382
Epoch 5/20
346/346 [==============================] - 119s 343ms/step - loss: 0.1315 - accuracy: 0.9487 - val_loss: 0.1147 - val_accuracy: 0.9573
Epoch 6/20
346/346 [==============================] - 118s 342ms/step - loss: 0.1233 - accuracy: 0.9492 - val_loss: 0.1305 - val_accuracy: 0.9454
Epoch 7/20
346/346 [==============================] - 118s 341ms/step - loss: 0.1403 - accuracy: 0.9473 - val_loss: 0.5263 - va

In [20]:
from keras.models import load_model
model.save('covid_disease_vgg16.h5')